<h1><center>Cookie Cats Mobile Games</center></h1>
<center>May 2024</center>
<center>Celine Ng</center>

a id="0"></a> <br>
# Table of Contents

1. [Project Introduction](#1)     
    1. [Libraries Installation](#2) 
    1. [Data Retrieval](#3)
    1. [Data Cleaning](#4)
1. [EDA](#5) 
    1. [Quick Visualization](#6)
        1. [Summary](#7)
1. [Assumption Checking](#7) 
1. [Create a Validation Set](#4)     
    1. [Remove target column](#5) 
1. [Feature Scaling](#6)     
1. [First Model](#8)     
    1. [Evaluation Metrics for Training set](#9)     
    1. [Evaluation Metrics for Validation set](#10)     
    1. [First Submission](#11) 
1. [Selecting Models](#12)       
    1. [Helper Functions to Try New Models](#13)      
    1. [Split to the Small Data for Evaluating Models Fast](#14)     
    1. [ML Models](#15)         
        1. [XGBoost](#16)             
            1. [Training](#17)

<a id="1"></a> <br>
# 1. Notebook Preparation

### Goal



<a id="2"></a> <br>
## 1.1 Libraries Installation

In [2]:
%%capture
%pip install -r requirements.txt

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp

<a id="3"></a> <br>
## 1.2 Data Retrieval

In [4]:
cookiecats = pd.read_csv('dataset/cookie_cats.csv')

In [5]:
cookiecats.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [6]:
cookiecats.shape

(90189, 5)